In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib.parse

# pages links

In [215]:
JumiaWebsite = 'https://www.jumia.com.eg/ar/mlp-men-s-jeans/?viewType=list&page='


# Fitch data from JUMIA

In [217]:
def jumiaJeans (website):
    
    product_img =[]
    free_shpping_ability=[]
    product_review =[]
    product_text =[]
    product_price = []
    product_old_price=[]
    product_scorce=[]
    product_link=[]

    for i in range(1,51):
        website = website+str(i)+'#catalog-listing'
        response = requests.get(website)
        response.status_code
        soup = BeautifulSoup(response.content,'html.parser')
        results = soup.find_all('a',{'class' : 'core'})
        for result in results:
            try:
                product_text.append(result.find('h3',{'class':'name'}).get_text())
            except:
                product_text.append('N/A')
            try:
                product_price.append(result.find('div',{'class':'prc'}).get_text())
            except:
                product_price.append('N/A')
            try:
                product_old_price.append(result.find('div',{'class':'old'}).get_text())
            except:
                product_old_price.append('N/A')
            try:
                product_review.append(result.find('div',{'class':'rev'}).get_text())
            except:
                product_review.append('N/A')
            try:
                product_img.append(result.find('img',{'class':'img'}).get('data-src'))
            except:
                product_img.append('N/A')
            try:
                product_scorce.append(result.find('div',{'class':'tag _mall _sm'}).get_text())
            except:
                product_scorce.append('N/A')
            try:
                free_shpping_ability.append(result.find('p',{'class':'shipp'}).get_text())
            except:
                free_shpping_ability.append('N/A')
            try:
                product_link.append(result.get('href'))
            except:
                product_link.append('N/A')  
                
    combinedLink(product_link)
    #save into dataframe
    product_df = pd.DataFrame({'product_text':product_text,'new_price':product_price,'old_price':product_old_price,'product_review':product_review,'product_img':product_img,'product_scorce':product_scorce,'free_shpping_ability':free_shpping_ability,'product_link':product_link_combined})    
    #clean data
    cleandf(product_df)

### comine links to the domain to be used

In [216]:
def combinedLink(product_link):
    product_link_combined=[]
    for link in product_link:
        product_link_combined.append(urllib.parse.urljoin(domain,link))

# cleaning data


In [226]:
def cleandf(product_df):
    product_df["new_price"] = product_df["new_price"].str.replace("جنيه ","")
    product_df["old_price"] = product_df["old_price"].str.replace("جنيه ","")
    product_df["product_scorce"] = product_df["product_scorce"].str.replace("المتجر الرسمي","Official store")
    product_df["free_shpping_ability"] = product_df["free_shpping_ability"].str.replace("مؤهل للشحن المجاني","free shipping")

In [218]:
jumiaJeans(JumiaWebsite)

# TEST

In [227]:
product_df.head(5)

,product_text,new_price,old_price,product_review,product_img,product_scorce,free_shpping_ability,product_link
0,American Eagle Original Straight Jean - Dark R...,550.00,"1,100.00",4.7 out of 5(40),https://eg.jumia.is/unsafe/fit-in/300x300/filt...,N/A,free shipping,https://www.jumia.com.eg/ar/american-eagle-ori...
1,Andora Full Buttoned Casual Denim Jacket - Lig...,299.00,"1,199.00",4.2 out of 5(71),https://eg.jumia.is/unsafe/fit-in/300x300/filt...,Official store,free shipping,https://www.jumia.com.eg/ar/andora-full-button...
2,"Andora Bundle Of 3 Solid Casual Jeans - Black,...",499.00,749.00,4 out of 5(43),https://eg.jumia.is/unsafe/fit-in/300x300/filt...,Official store,N/A,https://www.jumia.com.eg/ar/andora-bundle-of-3...
3,Andora Full Buttoned Casual Denim Jacket - Med...,299.00,"1,199.00",4 out of 5(50),https://eg.jumia.is/unsafe/fit-in/300x300/filt...,Official store,free shipping,https://www.jumia.com.eg/ar/andora-full-button...
4,RUCK&MAEL by Andora Casual Jeans - Dark Blue,149.00,550.00,3.3 out of 5(3),https://eg.jumia.is/unsafe/fit-in/300x300/filt...,Official store,N/A,https://www.jumia.com.eg/ar/ruckmael-by-andora...


# Save to cvs file

In [229]:
#product_df.to_csv('JumiaJeans.csv')

# connecting to database

In [38]:
import pyodbc 
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=ABDELKADER;'
                      'Database=Jumia;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [183]:
insert_stat = """
INSERT INTO Jeans_Info 
VALUES (?,?,?,?,?,?,?,?)
"""

In [202]:
for text in product_df.values:
    print(list(text))
    cursor.execute(insert_stat,list(text))
conn.commit()    

['American Eagle Original Straight Jean - Dark Reflection', '550.00', '1,100.00', '4.7 out of 5(40)', 'https://eg.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/58/032412/1.jpg?5111', 'N/A', 'free shipping', 'https://www.jumia.com.eg/ar/american-eagle-original-straight-jean-dark-reflection-21423085.html']
['Andora Full Buttoned Casual Denim Jacket - Light Blue ( Button color may be vary )', '299.00', '1,199.00', '4.2 out of 5(71)', 'https://eg.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/09/644822/1.jpg?4536', 'Official store', 'free shipping', 'https://www.jumia.com.eg/ar/andora-full-buttoned-casual-denim-jacket-light-blue-button-color-may-be-vary-22844690.html']
['Andora Bundle Of 3 Solid Casual Jeans - Black, Dark Blue & Navy Blue', '499.00', '749.00', '4 out of 5(43)', 'https://eg.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/71/038532/1.jpg?9486', 'Official store', 'N/A', 'https://www.jumia.com.eg/ar/andora-bundle-of-3-solid-casual-jeans-blac

In [ ]:
cursor.execute('SELECT * FROM Jeans_Info')
for i in cursor:
    print(i)